In [47]:
# delete this cell if working on Pycharm
!pip install Bio
!pip install import-ipynb

# New Section

In [48]:
!pip install wandb

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:

import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import wandb
from datetime import datetime

# so we can import utils notebook (delete if working on Pycharm), you might need to change it to your working directory path
%cd "/content/drive/MyDrive/ColabNotebooks"
import import_ipynb
import utils

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

/content/drive/MyDrive/ColabNotebooks
Num GPUs Available:  1


In [51]:

###############################################################################
#                                                                             #
#              Parameters you can change, but don't have to                   #
#                                                                             #
###############################################################################


# number of ResNet blocks for the first ResNet and the kernel size.
RESNET_1_BLOCKS = 3
RESNET_1_KERNEL_SIZE = 15
RESNET_1_KERNEL_NUM = 64

###############################################################################
#                                                                             #
#                        Parameters you need to choose                        #
#                                                                             #
###############################################################################



# number of ResNet blocks for the second ResNet, dilation list to repeat and the kernel size.

RESNET_2_BLOCKS = 1
RESNET_2_KERNEL_SIZE = 1  # good start may be 3/5
RESNET_2_KERNEL_NUM = 3  # DO NOT MAKE IT 1!
DILATION = [1]
WANTED_M = len(DILATION) # len of DILATION to be randomize by 'wandb' tool

# percentage of dropout for the dropout layer
DROPOUT = 0.0  # good start may be 0.1-0.5

# number of epochs, Learning rate and Batch size
EPOCHS = 3
LR = 0.001  # good start may be 0.0001/0.001/0.01
BATCH = 128  # good start may be 32/64/128


def get_time():
    now = datetime.now()
    return now.strftime("%d-%m-%Y__%H-%M-%S")


def resnet_block(input_layer, kernel_size, kernel_num, dialation=1):
    bn1 = layers.BatchNormalization()(input_layer)
    conv1d_layer1 = layers.Conv1D(kernel_num, kernel_size, padding='same', activation='relu',
                                  dilation_rate=dialation)(bn1)
    bn2 = layers.BatchNormalization()(conv1d_layer1)
    conv1d_layer2 = layers.Conv1D(kernel_num, kernel_size, padding='same', activation='relu',
                                  dilation_rate=dialation)(bn2)
    return layers.Add()([input_layer, conv1d_layer2])


def resnet_1(input_layer, block_num=RESNET_1_BLOCKS, kernel_size=RESNET_1_KERNEL_SIZE,
             kernel_num=RESNET_1_KERNEL_NUM):
    """
    ResNet layer - input -> BatchNormalization -> Conv1D -> Relu -> BatchNormalization -> Conv1D -> Relu -> Add
    :param input_layer: input layer for the ResNet
    :return: last layer of the ResNet
    """
    last_layer_output = input_layer

    for i in range(block_num):
        last_layer_output = resnet_block(last_layer_output, kernel_size, kernel_num)

    return last_layer_output


def resnet_2(input_layer, block_num=RESNET_2_BLOCKS, kernel_size=RESNET_2_KERNEL_SIZE,
             kernel_num=RESNET_2_KERNEL_NUM, dial_lst=DILATION):
    """
    Dilated ResNet layer - input -> BatchNormalization -> dilated Conv1D -> Relu -> BatchNormalization -> dilated Conv1D -> Relu -> Add
    :param input_layer: input layer for the ResNet
    :return: last layer of the ResNet
    """
    last_layer_output = input_layer

    for i in range(block_num):
        for d in dial_lst:
            last_layer_output = resnet_block(last_layer_output, kernel_size, kernel_num, d)

    return last_layer_output



In [52]:
def build_network(config):
    """
    builds the neural network architecture as shown in the exercise.
    :return: a Keras Model
    """

    # input, shape (NB_MAX_LENGTH,FEATURE_NUM)
    input_layer = tf.keras.Input(shape=(utils.NB_MAX_LENGTH, utils.FEATURE_NUM))

    # Conv1D -> shape = (NB_MAX_LENGTH, RESNET_1_KERNEL_NUM)
    conv1d_layer = layers.Conv1D(config['RESNET_1_KERNEL_NUM'], config['RESNET_1_KERNEL_SIZE'],
                                 padding='same')(input_layer)

    # first ResNet -> shape = (NB_MAX_LENGTH, RESNET_1_KERNEL_NUM)
    resnet_layer = resnet_1(conv1d_layer, config['RESNET_1_BLOCKS'], config['RESNET_1_KERNEL_SIZE'],
                            config['RESNET_1_KERNEL_NUM'])

    # Conv1D -> shape = (NB_MAX_LENGTH, RESNET_2_KERNEL_NUM)
    conv1d_layer = layers.Conv1D(config['RESNET_2_KERNEL_NUM'], config['RESNET_2_KERNEL_SIZE'],
                                 padding="same")(resnet_layer)

    # second ResNet -> shape = (NB_MAX_LENGTH, RESNET_2_KERNEL_NUM)
    resnet_layer = resnet_2(conv1d_layer, config['RESNET_2_BLOCKS'], config['RESNET_2_KERNEL_SIZE'],
                            config['RESNET_2_KERNEL_NUM'], config['DILATATION'])

    dp = layers.Dropout(config['DROPOUT'])(resnet_layer)
    conv1d_layer = layers.Conv1D(config['RESNET_2_KERNEL_NUM'] // 2, config['RESNET_2_KERNEL_SIZE'],
                                 padding="same",
                                 activation='elu')(dp)
    dense = layers.Dense(15)(conv1d_layer)

    return tf.keras.Model(input_layer, dense)


def plot_val_train_loss(history):
    """
    plots the train and validation loss of the model at each epoch, saves it in 'model_loss_history.png'
    :param history: history object (output of fit function)
    :return: None
    """
    ig, axes = plt.subplots(1, 1, figsize=(15, 3))
    axes.plot(history.history['loss'], label='Training loss')
    axes.plot(history.history['val_loss'], label='Validation loss')
    axes.legend()
    axes.set_title("Train and Val MSE loss")

    plt.savefig("/content/drive/MyDrive/Ex4Files/model_loss_history")  # TODO: you can change the path here


def get_config():
    sweep_config = {}
    sweep_config['method'] = 'random'
    sweep_config['metric'] = {'name': 'loss', 'goal': 'minimize'}

    sweep_config['name'] = f"BioEx4_{get_time()}"
    param_dict = {
        'RESNET_1_BLOCKS': {'distribution': 'int_uniform', 'min': 1, 'max': 10},
        'RESNET_1_KERNEL_SIZE': {'distribution': 'int_uniform', 'min': 1, 'max': 10},
        'RESNET_1_KERNEL_NUM': {'distribution': 'int_uniform', 'min': 5, 'max': 100},
        'RESNET_2_BLOCKS': {'distribution': 'int_uniform', 'min': 1, 'max': 10},
        'RESNET_2_KERNEL_SIZE': {'distribution': 'int_uniform', 'min': 1, 'max': 10},
        'RESNET_2_KERNEL_NUM': {'distribution': 'int_uniform', 'min': 5, 'max': 100},
        'DROPOUT': {'distribution': 'uniform', 'min': 0.001, 'max': 0.5},
        'EPOCHS': {'distribution': 'int_uniform', 'min': 2, 'max': 30},
        "LR": {'distribution': 'uniform', 'min': 0.001, 'max': 0.05},
        'BATCH': {'values': [16, 32, 64, 128, 256]},
        'DILATATION':{'values':[[1,2,4],[1],[1,2],[1,4],[1,2,4,8],[2,2,2]]}
    }

    sweep_config['parameters'] = param_dict
    return sweep_config


def get_default_config():
    sweep_config = {'RESNET_1_BLOCKS': RESNET_1_BLOCKS,
                    'RESNET_1_KERNEL_SIZE': RESNET_1_KERNEL_SIZE,
                    'RESNET_1_KERNEL_NUM': RESNET_1_KERNEL_NUM,
                    'RESNET_2_BLOCKS': RESNET_2_BLOCKS,
                    'RESNET_2_KERNEL_SIZE': RESNET_2_KERNEL_SIZE,
                    'RESNET_2_KERNEL_NUM': RESNET_2_KERNEL_NUM,
                    'DROPOUT': DROPOUT, 'EPOCHS': EPOCHS, "LR": LR,
                    'DILATATION':DILATION,
                    'BATCH': BATCH, 'method': 'random',
                    'metric': {'name': 'loss', 'goal': 'minimize'},
                    'name': f"BioEx4_{get_time()}"}

    return sweep_config

class WandbCallback(tf.keras.callbacks.Callback):
    def __init__(self, fold):
        super(WandbCallback, self).__init__()
        self.fold = fold
        
    def on_epoch_end(self, epoch, logs=None):
        wandb.log({'loss':logs['loss'],'val_loss':logs['val_loss'],'fold':
        self.fold,'epoch':epoch})
    


In [53]:
def train(config=None):
    if config is None:
        config = get_default_config()
    with wandb.init(config=config):

        # __________________________________________loading the data__________________________________________
        config = wandb.config
        input = np.load("train_input.npy")  # numpy array of shape (1974,NB_MAX_LENGTH,FEATURE_NUM) - data
        labels = np.load("train_labels.npy")  # numpy array of shape (1974,NB_MAX_LENGTH,OUTPUT_SIZE) - labels
        save_dir = "BestFits/"
        model_name = "test"
        fold_var = 1
        kf = KFold(n_splits=5)
        my_optimizer = tf.keras.optimizers.Adam(learning_rate=config['LR'])
        loss = np.zeros(5)

        for t_idx, v_idx in kf.split(input, labels):
            X_t, X_v = input[t_idx], input[v_idx]
            y_t, y_v = labels[t_idx], labels[v_idx]

            model = build_network(config)
            # ____________________________________________compiling___________________________________________

            model.compile(optimizer=my_optimizer, loss='mean_squared_error')

            # ________________________________________creating callbacks______________________________________
            checkpoint = tf.keras.callbacks.ModelCheckpoint(f"{save_dir}"
                                                            f"{model_name}"
                                                            f"{fold_var}.ckpt",
                                                            monitor='val_loss',
                                                            save_best_only=True, mode='max')

            callbacks_list = [checkpoint,WandbCallback(fold_var)]

            # ________________________________________fitting the model_______________________________________
            history = model.fit(X_t, y_t,
                                epochs=config['EPOCHS'],
                                callbacks=callbacks_list,
                                batch_size=config['BATCH'],
                                validation_data=(X_v, y_v))


            # ________________________________________evaluate the model______________________________________
            best_model = tf.keras.models.load_model(f"{save_dir}"
                                                    f"{model_name}"
                                                    f"{fold_var}.ckpt")
            loss[fold_var - 1] = best_model.evaluate(X_v, y_v)
            fold_var += 1
            tf.keras.backend.clear_session()

        wandb.log({'loss': np.mean(loss), "std": np.std(loss)})


In [ ]:
main()
    



Create sweep with ID: xx43bfy6
Sweep URL: https://wandb.ai/avishai-elma/BioColabEx4_1/sweeps/xx43bfy6


wandb: Agent Starting Run: ou1a4civ with config:
wandb: 	BATCH: 64
wandb: 	DILATION_0: 1
wandb: 	DROPOUT: 0.4301573503695044
wandb: 	EPOCHS: 18
wandb: 	LR: 0.04066810584943885
wandb: 	RESNET_1_BLOCKS: 10
wandb: 	RESNET_1_KERNEL_NUM: 9
wandb: 	RESNET_1_KERNEL_SIZE: 4
wandb: 	RESNET_2_BLOCKS: 6
wandb: 	RESNET_2_KERNEL_NUM: 82
wandb: 	RESNET_2_KERNEL_SIZE: 4


Epoch 1/18
25/25 [==============================] - 27s 716ms/step - loss: 302.1893 - val_loss: 6842272133939200.0000
Epoch 2/18
25/25 [==============================] - 1s 45ms/step - loss: 73.1749 - val_loss: 8116405760.0000
Epoch 3/18
25/25 [==============================] - 1s 42ms/step - loss: 63.2466 - val_loss: 340135.4062
Epoch 4/18
25/25 [==============================] - 1s 42ms/step - loss: 57.6689 - val_loss: 2993.9666
Epoch 5/18
25/25 [==============================] - 1s 42ms/step - loss: 55.2060 - val_loss: 53.1476
Epoch 6/18
25/25 [==============================] - 1s 42ms/step - loss: 54.1258 - val_loss: 52.0204
Epoch 7/18
25/25 [==============================] - 1s 42ms/step - loss: 53.7058 - val_loss: 51.7367
Epoch 8/18
25/25 [==============================] - 1s 42ms/step - loss: 53.4792 - val_loss: 51.6238
Epoch 9/18
25/25 [==============================] - 1s 42ms/step - loss: 53.4037 - val_loss: 51.5769
Epoch 10/18
25/25 [==============================] - 1s 42ms

loss,▁
std,▁
loss,1368467253394038.5
std,2736899219183602.0


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 87hwi4m9 with config:
wandb: 	BATCH: 256
wandb: 	DILATION_0: 2
wandb: 	DROPOUT: 0.041664229767928
wandb: 	EPOCHS: 3
wandb: 	LR: 0.02117408704267
wandb: 	RESNET_1_BLOCKS: 8
wandb: 	RESNET_1_KERNEL_NUM: 68
wandb: 	RESNET_1_KERNEL_SIZE: 6
wandb: 	RESNET_2_BLOCKS: 8
wandb: 	RESNET_2_KERNEL_NUM: 8
wandb: 	RESNET_2_KERNEL_SIZE: 2


...

wandb: Agent Starting Run: rl1rz24n with config:
wandb: 	BATCH: 128
wandb: 	DILATION_0: 5
wandb: 	DROPOUT: 0.2604626753217568
wandb: 	EPOCHS: 10
wandb: 	LR: 0.01663533991988174
wandb: 	RESNET_1_BLOCKS: 8
wandb: 	RESNET_1_KERNEL_NUM: 6
wandb: 	RESNET_1_KERNEL_SIZE: 5
wandb: 	RESNET_2_BLOCKS: 7
wandb: 	RESNET_2_KERNEL_NUM: 28
wandb: 	RESNET_2_KERNEL_SIZE: 1


Epoch 1/10
13/13 [==============================] - 25s 1s/step - loss: 44.9056 - val_loss: 145.2230
Epoch 2/10
13/13 [==============================] - 1s 48ms/step - loss: 22.2027 - val_loss: 114.8885
Epoch 3/10
13/13 [==============================] - 1s 43ms/step - loss: 14.5626 - val_loss: 96.3320
Epoch 4/10
13/13 [==============================] - 1s 45ms/step - loss: 8.8538 - val_loss: 63.1611
Epoch 5/10
13/13 [==============================] - 1s 44ms/step - loss: 6.2617 - val_loss: 54.8299
Epoch 6/10
13/13 [==============================] - 1s 45ms/step - loss: 5.3169 - val_loss: 38.9104
Epoch 7/10
13/13 [==============================] - 1s 46ms/step - loss: 4.9074 - val_loss: 36.4151
Epoch 8/10
13/13 [==============================] - 1s 47ms/step - loss: 4.6378 - val_loss: 29.0430
Epoch 9/10
13/13 [==============================] - 1s 45ms/step - loss: 4.5077 - val_loss: 22.8557
Epoch 10/10
13/13 [==============================] - 1s 10ms/step - loss: 145.2230
Epoch 1/10
13

loss,▁
std,▁
loss,2481.02831
std,4517.13073


wandb: Agent Starting Run: medm6pyy with config:
wandb: 	BATCH: 16
wandb: 	DILATION_0: 5
wandb: 	DROPOUT: 0.05405608459480533
wandb: 	EPOCHS: 21
wandb: 	LR: 0.02194479094582053
wandb: 	RESNET_1_BLOCKS: 4
wandb: 	RESNET_1_KERNEL_NUM: 5
wandb: 	RESNET_1_KERNEL_SIZE: 2
wandb: 	RESNET_2_BLOCKS: 9
wandb: 	RESNET_2_KERNEL_NUM: 85
wandb: 	RESNET_2_KERNEL_SIZE: 8


Epoch 1/21
99/99 [==============================] - 26s 167ms/step - loss: 82.8258 - val_loss: 51.5823
Epoch 2/21
99/99 [==============================] - 4s 37ms/step - loss: 53.3457 - val_loss: 51.5530
Epoch 3/21
99/99 [==============================] - 4s 36ms/step - loss: 53.3444 - val_loss: 51.5497
Epoch 4/21
99/99 [==============================] - 4s 36ms/step - loss: 53.3454 - val_loss: 51.5533
Epoch 5/21
99/99 [==============================] - 4s 36ms/step - loss: 53.3449 - val_loss: 51.5513
Epoch 6/21
99/99 [==============================] - 4s 37ms/step - loss: 53.3458 - val_loss: 51.5575
Epoch 7/21
99/99 [==============================] - 4s 36ms/step - loss: 53.3451 - val_loss: 51.5506
Epoch 8/21
99/99 [==============================] - 4s 36ms/step - loss: 53.3463 - val_loss: 51.5614
Epoch 9/21
99/99 [==============================] - 4s 35ms/step - loss: 53.3457 - val_loss: 51.5524
Epoch 10/21
99/99 [==============================] - 4s 36ms/step - loss: 53.3459 - val_l